In [1]:
"""
Thermal Human Detection using YOLOv8 - Jupyter Notebook Version
================================================================
A comprehensive system for detecting humans in thermal imagery using YOLOv8.

Author: [Your Name]
Date: December 2025
"""

# ============================================================================
# Install Required Packages
# ============================================================================
# Run this cell first to install all dependencies

"""
!pip install ultralytics supervision huggingface_hub opencv-python torch pandas matplotlib seaborn
"""

'\n!pip install ultralytics supervision huggingface_hub opencv-python torch pandas matplotlib seaborn\n'

In [2]:
# ============================================================================
# Import Libraries
# ============================================================================

from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
import supervision as sv
import cv2
import torch
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from IPython.display import display, Image, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
print(f"✓ Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

✓ All libraries imported successfully!
✓ PyTorch version: 2.9.1+cu130
✓ CUDA available: True
✓ Device: cuda


In [3]:
# ============================================================================
# Configuration
# ============================================================================

class Config:
    """Configuration parameters for the project"""
    
    # Model settings
    MODEL_REPO = "pitangent-ds/YOLOv8-human-detection-thermal"
    MODEL_FILENAME = "model.pt"
    BASE_MODEL = "yolov8n.pt"
    
    # Training hyperparameters
    BATCH_SIZE = 16  # Reduced for notebooks
    EPOCHS = 30
    IMG_SIZE = [640, 480]
    OPTIMIZER = "AdamW"
    LEARNING_RATE = 3e-5
    WARMUP_EPOCHS = 10
    CONFIDENCE_THRESHOLD = 0.6
    
    # Data augmentation
    AUGMENTATION = {
        "hsv_h": 0.015,
        "hsv_s": 0.7,
        "hsv_v": 0.4,
        "degrees": 10.0,
        "translate": 0.1,
        "scale": 0.5,
        "flipud": 0.5,
        "fliplr": 0.5,
        "mosaic": 1.0,
        "mixup": 0.1,
    }
    
    # Paths
    DATA_YAML = "data.yaml"
    OUTPUT_DIR = "outputs"

# Create output directories
Path(Config.OUTPUT_DIR).mkdir(exist_ok=True)
Path(f"{Config.OUTPUT_DIR}/images").mkdir(exist_ok=True)
Path(f"{Config.OUTPUT_DIR}/videos").mkdir(exist_ok=True)
Path(f"{Config.OUTPUT_DIR}/plots").mkdir(exist_ok=True)

print("✓ Configuration loaded successfully!")
print(f"✓ Output directory: {Config.OUTPUT_DIR}")

✓ Configuration loaded successfully!
✓ Output directory: outputs


In [4]:
# ============================================================================
# Download Pre-trained Model (Optional)
# ============================================================================

def download_pretrained_model():
    """Download pre-trained thermal detection model from HuggingFace"""
    print("Downloading pre-trained model from HuggingFace...")
    try:
        model_path = hf_hub_download(
            repo_id=Config.MODEL_REPO,
            filename=Config.MODEL_FILENAME
        )
        print(f"✓ Model downloaded to: {model_path}")
        return model_path
    except Exception as e:
        print(f"✗ Error downloading model: {e}")
        return None

# Uncomment to download pre-trained model
# pretrained_path = download_pretrained_model()
# pretrained_model = YOLO(pretrained_path)

print("Ready to download pre-trained model (uncomment code above)")

Ready to download pre-trained model (uncomment code above)


In [5]:
# ============================================================================
# Load Base Model
# ============================================================================

print("Loading YOLOv8 base model...")
model = YOLO(Config.BASE_MODEL)
print(f"✓ Loaded model: {Config.BASE_MODEL}")
print(f"✓ Model summary:")
print(model.info())

Loading YOLOv8 base model...
✓ Loaded model: yolov8n.pt
✓ Model summary:
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
(129, 3157200, 0, 8.8575488)


In [6]:
# ============================================================================
# Train Model
# ============================================================================

def train_model(model, data_yaml=Config.DATA_YAML, epochs=Config.EPOCHS):
    """
    Train YOLOv8 model on thermal dataset
    """
    print("=" * 70)
    print("STARTING MODEL TRAINING")
    print("=" * 70)
    
    # Training hyperparameters
    hyperparams = {
        "batch": Config.BATCH_SIZE,
        "epochs": epochs,
        "imgsz": Config.IMG_SIZE,
        "optimizer": Config.OPTIMIZER,
        "cos_lr": True,
        "lr0": Config.LEARNING_RATE,
        "warmup_epochs": Config.WARMUP_EPOCHS,
    }
    
    # Combine with augmentation parameters
    training_params = {**hyperparams, **Config.AUGMENTATION}
    
    print("\nTraining Configuration:")
    print("-" * 50)
    for key, value in training_params.items():
        print(f"  {key}: {value}")
    print("-" * 50)
    
    # Start training
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\n✓ Training on device: {device}")
    
    results = model.train(
        device=device,
        data=data_yaml,
        **training_params,
        plots=True,  # Generate plots
        save=True,   # Save checkpoints
        verbose=True
    )
    
    print("\n" + "=" * 70)
    print("✓ TRAINING COMPLETE!")
    print("=" * 70)
    
    return model, results

# Uncomment and modify data_yaml path to train
trained_model, results = train_model(model, data_yaml="data.yaml", epochs=30)

print("Ready to train model (uncomment code above and set data.yaml path)")

STARTING MODEL TRAINING

Training Configuration:
--------------------------------------------------
  batch: 16
  epochs: 30
  imgsz: [640, 480]
  optimizer: AdamW
  cos_lr: True
  lr0: 3e-05
  warmup_epochs: 10
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  degrees: 10.0
  translate: 0.1
  scale: 0.5
  flipud: 0.5
  fliplr: 0.5
  mosaic: 1.0
  mixup: 0.1
--------------------------------------------------

✓ Training on device: cuda
Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060, 11909MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript

In [7]:
# ============================================================================
# Evaluation Functions
# ============================================================================

def evaluate_model(model, data_yaml=Config.DATA_YAML):
    """Evaluate model performance"""
    print("=" * 70)
    print("MODEL EVALUATION")
    print("=" * 70)
    
    results = model.val(data=data_yaml)
    
    print("\n📊 Performance Metrics:")
    print("-" * 50)
    print(f"  mAP@50:     {results.box.map50:.4f}")
    print(f"  mAP@50-95:  {results.box.map:.4f}")
    print(f"  Precision:  {results.box.p:.4f}")
    print(f"  Recall:     {results.box.r:.4f}")
    print("-" * 50)
    
    return results

# Uncomment to evaluate
# eval_results = evaluate_model(model, data_yaml="path/to/data.yaml")

print("Ready to evaluate model")

Ready to evaluate model


In [8]:
# ============================================================================
# Single Image Inference
# ============================================================================

def inference_single_image(image_path, model, conf=Config.CONFIDENCE_THRESHOLD):
    """
    Perform inference on a single image and display results
    """
    print(f"Processing image: {image_path}")
    
    # Read image
    cv_image = cv2.imread(image_path)
    if cv_image is None:
        print(f"✗ Error: Could not read image at {image_path}")
        return None
    
    # Perform detection
    results = model(cv_image, conf=conf, verbose=False)
    detections = Detections.from_ultralytics(results[0])
    
    print(f"✓ Detected {len(detections)} humans")
    
    # Create annotators
    box_annotator = sv.BoxAnnotator(thickness=2, color=sv.Color.GREEN)
    label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=0.5)
    
    # Create labels
    labels = [f"Person {conf:.2f}" for conf in detections.confidence]
    
    # Annotate
    annotated = box_annotator.annotate(scene=cv_image.copy(), detections=detections)
    annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)
    
    # Add count
    cv2.putText(annotated, f"Humans: {len(detections)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Convert BGR to RGB for display
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    # Display
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_rgb)
    plt.axis('off')
    plt.title(f'Detection Results - {len(detections)} Humans Detected')
    plt.tight_layout()
    plt.show()
    
    # Save
    output_path = f"{Config.OUTPUT_DIR}/images/result_{Path(image_path).stem}.jpg"
    cv2.imwrite(output_path, annotated)
    print(f"✓ Saved result to: {output_path}")
    
    return detections, annotated

# Example usage (replace with your image path):
# detections, annotated = inference_single_image("path/to/your/image.jpg", model)

print("Ready for single image inference")

Ready for single image inference


In [9]:
# ============================================================================
# Batch Image Inference
# ============================================================================

def inference_batch_images(image_folder, model, conf=Config.CONFIDENCE_THRESHOLD):
    """
    Process multiple images from a folder
    """
    image_paths = list(Path(image_folder).glob("*.jpg")) + \
                  list(Path(image_folder).glob("*.png"))
    
    print(f"Found {len(image_paths)} images in {image_folder}")
    
    results_list = []
    
    for i, img_path in enumerate(image_paths):
        print(f"\nProcessing {i+1}/{len(image_paths)}: {img_path.name}")
        detections, _ = inference_single_image(str(img_path), model, conf)
        results_list.append({
            'image': img_path.name,
            'detections': len(detections)
        })
    
    # Summary
    df = pd.DataFrame(results_list)
    print("\n" + "=" * 50)
    print("BATCH PROCESSING SUMMARY")
    print("=" * 50)
    print(df.to_string(index=False))
    print(f"\nTotal humans detected: {df['detections'].sum()}")
    print(f"Average per image: {df['detections'].mean():.2f}")
    
    return df

# Example usage:
# batch_results = inference_batch_images("path/to/image/folder", model)

print("Ready for batch image inference")

Ready for batch image inference


In [10]:
# ============================================================================
# Video Processing
# ============================================================================

def process_video(video_path, model, output_path=None, conf=Config.CONFIDENCE_THRESHOLD, 
                  display_every=30):
    """
    Process video for human detection
    """
    if output_path is None:
        output_path = f"{Config.OUTPUT_DIR}/videos/output_{Path(video_path).stem}.mp4"
    
    print(f"📹 Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"✗ Error: Could not open video")
        return None
    
    # Video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video info: {width}x{height} @ {fps} FPS, {total_frames} frames")
    
    # Output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    box_annotator = sv.BoxAnnotator(thickness=2, color=sv.Color.GREEN)
    
    frame_count = 0
    detection_counts = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect
        results = model(frame, conf=conf, verbose=False)
        detections = Detections.from_ultralytics(results[0])
        
        # Annotate
        annotated = box_annotator.annotate(scene=frame.copy(), detections=detections)
        
        # Add info
        cv2.putText(annotated, f"Humans: {len(detections)}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(annotated, f"Frame: {frame_count+1}/{total_frames}", (10, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        out.write(annotated)
        detection_counts.append(len(detections))
        frame_count += 1
        
        # Progress update
        if frame_count % display_every == 0:
            clear_output(wait=True)
            print(f"Progress: {frame_count}/{total_frames} frames ({frame_count/total_frames*100:.1f}%)")
            print(f"Current detections: {len(detections)}")
    
    cap.release()
    out.release()
    
    print(f"\n✓ Video processing complete!")
    print(f"✓ Output saved to: {output_path}")
    print(f"✓ Average humans per frame: {np.mean(detection_counts):.2f}")
    
    return detection_counts

# Example usage:
detection_counts = process_video("test.mp4", model)

print("Ready for video processing")

Progress: 4710/4720 frames (99.8%)
Current detections: 1

✓ Video processing complete!
✓ Output saved to: outputs/videos/output_test.mp4
✓ Average humans per frame: 4.09
Ready for video processing


In [11]:
# ============================================================================
# Plot Training History
# ============================================================================

def plot_training_history(csv_path='runs/detect/train/results.csv'):
    """Visualize training metrics"""
    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Training History', fontsize=16, fontweight='bold')
        
        # Box Loss
        axes[0, 0].plot(df['train/box_loss'], label='Train', linewidth=2, color='blue')
        axes[0, 0].set_title('Box Loss', fontweight='bold')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Class Loss
        axes[0, 1].plot(df['train/cls_loss'], label='Train', linewidth=2, color='orange')
        axes[0, 1].set_title('Classification Loss', fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # mAP@50
        axes[1, 0].plot(df['metrics/mAP50(B)'], label='mAP@50', linewidth=2, color='green')
        axes[1, 0].set_title('mAP@50', fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('mAP')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Precision & Recall
        axes[1, 1].plot(df['metrics/precision(B)'], label='Precision', linewidth=2, color='blue')
        axes[1, 1].plot(df['metrics/recall(B)'], label='Recall', linewidth=2, color='red')
        axes[1, 1].set_title('Precision & Recall', fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Score')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        output_path = f"{Config.OUTPUT_DIR}/plots/training_history.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✓ Plot saved to: {output_path}")
        
    except FileNotFoundError:
        print(f"✗ Results file not found at {csv_path}")
        print("Train the model first!")
    except Exception as e:
        print(f"✗ Error: {e}")

# Uncomment after training:
# plot_training_history()

print("Ready to plot training history")

Ready to plot training history


In [12]:
# ============================================================================
# Plot Video Detections
# ============================================================================

def plot_video_detections(detection_counts, output_path=None):
    """Plot detection counts over video frames"""
    if output_path is None:
        output_path = f"{Config.OUTPUT_DIR}/plots/video_detections.png"
    
    plt.figure(figsize=(14, 6))
    plt.plot(detection_counts, linewidth=2, color='blue', alpha=0.7)
    plt.fill_between(range(len(detection_counts)), detection_counts, alpha=0.3, color='blue')
    
    # Statistics
    avg = np.mean(detection_counts)
    max_det = np.max(detection_counts)
    
    plt.axhline(y=avg, color='red', linestyle='--', linewidth=2, 
                label=f'Average: {avg:.2f}')
    plt.axhline(y=max_det, color='green', linestyle='--', linewidth=2, 
                label=f'Maximum: {max_det}')
    
    plt.title('Human Detections Over Video Frames', fontsize=14, fontweight='bold')
    plt.xlabel('Frame Number', fontsize=12)
    plt.ylabel('Number of Humans Detected', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Plot saved to: {output_path}")

# Use after processing video:
# plot_video_detections(detection_counts)

print("Ready to plot video detections")

Ready to plot video detections


In [13]:
# ============================================================================
# Model Comparison
# ============================================================================

def compare_models(model_names=['yolov8n.pt', 'yolov8s.pt'], data_yaml=Config.DATA_YAML):
    """Compare different YOLO model variants"""
    print("=" * 70)
    print("MODEL COMPARISON")
    print("=" * 70)
    
    results = {}
    
    for model_name in model_names:
        print(f"\n📊 Evaluating {model_name}...")
        temp_model = YOLO(model_name)
        val_results = temp_model.val(data=data_yaml, verbose=False)
        
        results[model_name] = {
            'mAP@50': val_results.box.map50,
            'mAP@50-95': val_results.box.map,
            'Precision': val_results.box.p,
            'Recall': val_results.box.r,
            'Inference (ms)': val_results.speed['inference']
        }
    
    df = pd.DataFrame(results).T
    
    print("\n" + "=" * 70)
    print("COMPARISON RESULTS")
    print("=" * 70)
    print(df.to_string())
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # mAP comparison
    df[['mAP@50', 'mAP@50-95']].plot(kind='bar', ax=axes[0], color=['green', 'blue'])
    axes[0].set_title('mAP Comparison', fontweight='bold')
    axes[0].set_ylabel('Score')
    axes[0].set_xlabel('Model')
    axes[0].legend(['mAP@50', 'mAP@50-95'])
    axes[0].grid(True, alpha=0.3)
    
    # Speed comparison
    df['Inference (ms)'].plot(kind='bar', ax=axes[1], color='orange')
    axes[1].set_title('Inference Speed', fontweight='bold')
    axes[1].set_ylabel('Time (ms)')
    axes[1].set_xlabel('Model')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{Config.OUTPUT_DIR}/plots/model_comparison.png", dpi=300)
    plt.show()
    
    # Save CSV
    df.to_csv(f"{Config.OUTPUT_DIR}/model_comparison.csv")
    print(f"\n✓ Saved to: {Config.OUTPUT_DIR}/model_comparison.csv")
    
    return df

# Uncomment to compare:
# comparison_df = compare_models(data_yaml="path/to/data.yaml")

print("Ready for model comparison")

Ready for model comparison


In [14]:
# ============================================================================
# CELL 14: Benchmark Model Speed
# ============================================================================

def benchmark_model(model, image_path, num_runs=100):
    """Benchmark inference speed"""
    print("=" * 70)
    print("MODEL BENCHMARKING")
    print("=" * 70)
    
    cv_image = cv2.imread(image_path)
    if cv_image is None:
        print(f"✗ Error reading image: {image_path}")
        return None
    
    print(f"Running {num_runs} iterations...")
    
    # Warmup
    for _ in range(10):
        model(cv_image, conf=Config.CONFIDENCE_THRESHOLD, verbose=False)
    
    # Benchmark
    times = []
    for i in range(num_runs):
        start = time.time()
        model(cv_image, conf=Config.CONFIDENCE_THRESHOLD, verbose=False)
        times.append(time.time() - start)
        
        if (i + 1) % 20 == 0:
            clear_output(wait=True)
            print(f"Progress: {i+1}/{num_runs}")
    
    clear_output(wait=True)
    
    # Statistics
    avg_time = np.mean(times)
    std_time = np.std(times)
    min_time = np.min(times)
    max_time = np.max(times)
    fps = 1 / avg_time
    
    print("\n📊 Benchmark Results:")
    print("-" * 50)
    print(f"  Average time:  {avg_time*1000:.2f} ms")
    print(f"  Std deviation: {std_time*1000:.2f} ms")
    print(f"  Min time:      {min_time*1000:.2f} ms")
    print(f"  Max time:      {max_time*1000:.2f} ms")
    print(f"  FPS:           {fps:.2f}")
    print("-" * 50)
    
    # Plot distribution
    plt.figure(figsize=(10, 5))
    plt.hist(np.array(times) * 1000, bins=30, color='blue', alpha=0.7, edgecolor='black')
    plt.axvline(avg_time * 1000, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {avg_time*1000:.2f} ms')
    plt.title('Inference Time Distribution', fontweight='bold')
    plt.xlabel('Time (ms)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{Config.OUTPUT_DIR}/plots/benchmark.png", dpi=300)
    plt.show()
    
    return {'avg_ms': avg_time * 1000, 'fps': fps}

# Example:
# benchmark_results = benchmark_model(model, "path/to/test/image.jpg")

print("Ready for benchmarking")

Ready for benchmarking


In [15]:
# ============================================================================
# CELL 15: Export Model
# ============================================================================

def export_model(model, formats=['onnx']):
    """Export model for deployment"""
    print("=" * 70)
    print("MODEL EXPORT")
    print("=" * 70)
    
    for fmt in formats:
        print(f"\n📦 Exporting to {fmt.upper()}...")
        try:
            model.export(format=fmt)
            print(f"✓ Successfully exported to {fmt}")
        except Exception as e:
            print(f"✗ Export failed: {e}")
    
    print("\n✓ Export complete!")

# Uncomment to export:
# export_model(model, formats=['onnx', 'torchscript'])

print("Ready to export model")

Ready to export model
